In [3]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [5]:
# df_solar_new = pd.read_csv("//srvditz1/home$/hohensteinpa/Desktop/Test/Energy_production_price_prediction/basic_files/solar_total_production.csv")
# df_solar_new_forecast= pd.read_csv("//srvditz1/home$/hohensteinpa/Desktop/Test/Energy_production_price_prediction/basic_files/solar_wind_forecast.csv") 

# df_solar_new = pd.read_csv("D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv")
# df_solar_new_forecast= pd.read_csv("D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_wind_forecast.csv") 
df_solar_new = pd.read_csv("C:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv")
df_solar_new_forecast= pd.read_csv("C:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_wind_forecast.csv") 

In [43]:
merged_df = pd.merge(df_solar_new_forecast, df_solar_new, on='timestamp_utc', how='inner')

In [44]:
merged_df["solar_diff"] = merged_df.generation_mw - merged_df.solar_mw

In [ ]:
merged_df.solar_diff.describe()

In [7]:
# df_solar = pd.read_csv("//srvditz1/home$/hohensteinpa/Desktop/Test/Energy_production_price_prediction/HEFTcom24/data/solar1.csv")
df_solar = pd.read_csv("C:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/HEFTcom24/data/solar1.csv")

In [8]:
df_solar.drop(columns=['Unnamed: 0','boa_MWh','Wind_MW','Wind_MWh_credit'], inplace=True)
df_solar.valid_time = pd.to_datetime(df_solar.valid_time) 
df_solar.reference_time = pd.to_datetime(df_solar.reference_time)
df_solar = df_solar.groupby("valid_time").last().reset_index()

In [49]:
df_solar[[f'Temperature_Point{i}_dwd' for i in range(7)]].std()

Temperature_Point0_dwd    6.228025
Temperature_Point1_dwd    5.840663
Temperature_Point2_dwd    6.302572
Temperature_Point3_dwd    6.219771
Temperature_Point4_dwd    6.170284
Temperature_Point5_dwd    5.919253
Temperature_Point6_dwd    5.910956
dtype: float64

In [ ]:
df_solar.groupby("valid_time").count()

In [9]:
# Extracting hour of the day from 'valid_time'
df_solar['hour'] = df_solar['valid_time'].dt.hour

# Sine and cosine encoding for hour (for cyclical behavior)
df_solar['sin_hour'] = np.sin(2 * np.pi * df_solar['hour'] / 24)
df_solar['cos_hour'] = np.cos(2 * np.pi * df_solar['hour'] / 24)

# Day of the year (seasonality)
df_solar['day_of_year'] = df_solar['valid_time'].dt.dayofyear

# Sine and cosine encoding for day of the year (for cyclical seasonality)
df_solar['sin_day'] = np.sin(2 * np.pi * df_solar['day_of_year'] / 365)
df_solar['cos_day'] = np.cos(2 * np.pi * df_solar['day_of_year'] / 365)
df_solar['Mean_SolarRadiation_dwd'] = df_solar[[f'SolarDownwardRadiation_Point{i}_dwd' for i in range(20)]].mean(axis=1)
df_solar['Mean_Temperature_dwd'] = df_solar[[f'Temperature_Point{i}_dwd' for i in range(7)]].mean(axis=1)
df_solar['Std_Temperature_dwd'] = df_solar[[f'Temperature_Point{i}_dwd' for i in range(7)]].std(axis=1)
df_solar["SolarDownwardRadiation_RW_dwd_Mean_30min"] = df_solar["Mean_SolarRadiation_dwd"].rolling(window= 1).mean()
df_solar["SolarDownwardRadiation_RW_dwd_Mean_1h"] = df_solar["Mean_SolarRadiation_dwd"].rolling(window= 2).mean()
df_solar["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df_solar["Mean_SolarRadiation_dwd"].shift(periods= 1)
df_solar["SolarDownwardRadiation_dwd_Mean_Lag_1h"] = df_solar["Mean_SolarRadiation_dwd"].shift(periods= 2)
df_solar["SolarDownwardRadiation_dwd_Mean_Lag_24h"] = df_solar["Mean_SolarRadiation_dwd"].shift(periods= 48)


In [27]:
df_solar.SolarDownwardRadiation_dwd_Mean_Lag_24h

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
50930    0.004615
50931    0.017088
50932   -0.000814
50933   -0.018716
50934   -0.006566
Name: SolarDownwardRadiation_dwd_Mean_Lag_24h, Length: 50935, dtype: float64

In [10]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    # Calculate cell temperature using the simplified NOCT model
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    # Calculate the efficiency loss due to increased cell temperature
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [11]:
temperature_columns = [
 'Temperature_Point0_dwd',
 'Temperature_Point1_dwd',
 'Temperature_Point2_dwd',
 'Temperature_Point3_dwd',
 'Temperature_Point4_dwd',
 'Temperature_Point5_dwd',
 'Temperature_Point6_dwd',
 'Temperature_Point7_dwd',
 'Temperature_Point8_dwd',
 'Temperature_Point9_dwd',
 'Temperature_Point10_dwd',
 'Temperature_Point11_dwd',
 'Temperature_Point12_dwd',
 'Temperature_Point13_dwd',
 'Temperature_Point14_dwd',
 'Temperature_Point15_dwd',
 'Temperature_Point16_dwd',
 'Temperature_Point17_dwd',
 'Temperature_Point18_dwd',
 'Temperature_Point19_dwd',
]

irradiance_columns = [
    'SolarDownwardRadiation_Point0_dwd', 'SolarDownwardRadiation_Point1_dwd',
    'SolarDownwardRadiation_Point2_dwd', 'SolarDownwardRadiation_Point3_dwd',
    'SolarDownwardRadiation_Point4_dwd', 'SolarDownwardRadiation_Point5_dwd',
    'SolarDownwardRadiation_Point6_dwd', 'SolarDownwardRadiation_Point7_dwd',
    'SolarDownwardRadiation_Point8_dwd', 'SolarDownwardRadiation_Point9_dwd',
    'SolarDownwardRadiation_Point10_dwd', 'SolarDownwardRadiation_Point11_dwd',
    'SolarDownwardRadiation_Point12_dwd', 'SolarDownwardRadiation_Point13_dwd',
    'SolarDownwardRadiation_Point14_dwd', 'SolarDownwardRadiation_Point15_dwd',
    'SolarDownwardRadiation_Point16_dwd', 'SolarDownwardRadiation_Point17_dwd',
    'SolarDownwardRadiation_Point18_dwd', 'SolarDownwardRadiation_Point19_dwd',
]
for i in range(20):
    for source in ['dwd']:
        temp_col = f'Temperature_Point{i}_{source}'
        irradiance_col = f'SolarDownwardRadiation_Point{i}_{source}'
        panel_temp_col = f'Panel_Temperature_Point{i}_{source}'
        panel_eff_col = f'Panel_Efficiency_Point{i}_{source}'
        
        df_solar[panel_temp_col], df_solar[panel_eff_col] = zip(*df_solar.apply(
            lambda row: pv_temperature_efficiency(row[irradiance_col], row[temp_col]), axis=1))

In [12]:
df_solar["Panel_Temperature_dwd_mean"] = df_solar.filter(regex= r"Panel_Temperature.*_dwd").mean(axis= 1)
df_solar["Panel_Efficiency_dwd_mean"] = df_solar.filter(regex= r"Panel_Efficiency.*_dwd").mean(axis= 1)
df_solar["Panel_Temperature_dwd_std"] = df_solar.filter(regex= r"Panel_Temperature.*_dwd").std(axis= 1)
df_solar["Panel_Efficiency_dwd_std"] = df_solar.filter(regex= r"Panel_Efficiency.*_dwd").std(axis= 1)

In [9]:
df_solar_new.columns.to_list()

['timestamp_utc', 'generation_mw', 'installed_capacity_mwp', 'capacity_mwp']

In [13]:
df_solar_new.timestamp_utc = pd.to_datetime(df_solar_new.timestamp_utc)

In [14]:
df_solar_new_forecast.timestamp_utc = pd.to_datetime(df_solar_new_forecast.timestamp_utc) 
merged_df = pd.merge(df_solar_new, df_solar, left_on='timestamp_utc',right_on='valid_time', how='inner')

In [15]:
merged_df.SolarDownwardRadiation_RW_dwd_Mean_1h

0             NaN
1        0.000000
2        0.000000
3        0.000000
4        0.000000
           ...   
50930    0.004048
50931    0.015587
50932    0.017116
50933    0.008635
50934    0.014732
Name: SolarDownwardRadiation_RW_dwd_Mean_1h, Length: 50935, dtype: float64

In [16]:
merged_df["solar_mw_lag_24h"] = merged_df.generation_mw.shift(periods= 48)

In [17]:
df_solar1 = merged_df[[ 
    "Mean_SolarRadiation_dwd",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_RW_dwd_Mean_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_24h",
    "Panel_Efficiency_dwd_mean",
    "Panel_Efficiency_dwd_std",
    "Panel_Temperature_dwd_mean",
    "Panel_Temperature_dwd_std",
    "Std_Temperature_dwd",
    "Mean_Temperature_dwd",
    "cos_hour",
    "cos_day","solar_mw_lag_24h","Solar_MWh_credit"]]

In [18]:
df_solar1.columns.shape

(16,)

In [19]:
df_solar1 = df_solar1.dropna(axis=0)

In [20]:
df_solar1.columns

Index(['Mean_SolarRadiation_dwd', 'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_RW_dwd_Mean_1h',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_1h',
       'SolarDownwardRadiation_dwd_Mean_Lag_24h', 'Panel_Efficiency_dwd_mean',
       'Panel_Efficiency_dwd_std', 'Panel_Temperature_dwd_mean',
       'Panel_Temperature_dwd_std', 'Std_Temperature_dwd',
       'Mean_Temperature_dwd', 'cos_hour', 'cos_day', 'solar_mw_lag_24h',
       'Solar_MWh_credit'],
      dtype='object')

In [21]:
df_solar1.drop(columns=["Solar_MWh_credit"]).values.shape

(50887, 15)

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Assuming df_solar1 is already defined
# df_solar1 = ...

# Split the data into features and target
X = df_solar1.drop(columns=["Solar_MWh_credit"]).values
y = df_solar1["Solar_MWh_credit"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# save the scaler
import pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [24]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 64)
        self.fc5 = nn.Linear(64, 9)

        self.dropout = nn.Dropout(0.2)
        self.swish = nn.SiLU()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.swish(self.fc2(x))
        x = self.dropout(x)
        x = self.swish(self.fc3(x))
        x = self.dropout(x)
        x = self.swish(self.fc4(x))
        x = self.dropout(x)
        x = self.fc5(x)
        return x

In [25]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)



# Initialize the model, loss function, and optimizer
input_dim = X_train_scaled.shape[1]
model = MLP(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [26]:
def pinball_loss(y_true, y_pred, quantiles):
    errors = y_true - y_pred
    loss = torch.max((quantiles - 1) * errors, quantiles * errors)
    return torch.mean(loss)

In [27]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    print("CUDA is available! GPU is ready for use.")
    print(f"GPU Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Running on CPU.")

CUDA is not available. Running on CPU.


In [28]:
import torch.optim as optim
quantiles = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

num_epochs = 500
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = pinball_loss(y_batch, outputs, quantiles)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    train_losses.append(epoch_loss / len(train_loader))
    
    # Evaluate the model on the test set
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        test_loss = pinball_loss(y_test_tensor, y_pred, quantiles)
        test_losses.append(test_loss.item())
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}")


Epoch [1/500], Train Loss: 59.5438, Test Loss: 60.0771
Epoch [2/500], Train Loss: 57.0327, Test Loss: 52.3555
Epoch [3/500], Train Loss: 36.0112, Test Loss: 20.1099
Epoch [4/500], Train Loss: 17.8601, Test Loss: 14.6763
Epoch [5/500], Train Loss: 14.9503, Test Loss: 12.5440
Epoch [6/500], Train Loss: 13.4527, Test Loss: 11.1792
Epoch [7/500], Train Loss: 12.6806, Test Loss: 10.5473
Epoch [8/500], Train Loss: 12.3428, Test Loss: 10.2777
Epoch [9/500], Train Loss: 12.0873, Test Loss: 10.0753
Epoch [10/500], Train Loss: 11.8891, Test Loss: 9.9164
Epoch [11/500], Train Loss: 11.8095, Test Loss: 9.8312
Epoch [12/500], Train Loss: 11.7857, Test Loss: 9.7834
Epoch [13/500], Train Loss: 11.6435, Test Loss: 9.6864
Epoch [14/500], Train Loss: 11.5855, Test Loss: 9.6257
Epoch [15/500], Train Loss: 11.4790, Test Loss: 9.5618
Epoch [16/500], Train Loss: 11.3134, Test Loss: 9.5070
Epoch [17/500], Train Loss: 11.3522, Test Loss: 9.4414
Epoch [18/500], Train Loss: 11.1786, Test Loss: 9.3867
Epoch [19/

In [29]:
import torch

# Assuming your trained model is called 'model'

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation (since we're only making predictions)
with torch.no_grad():
    # Get model predictions for X_test_tensor
    y_pred_tensor = model(X_test_tensor)  # Shape: (num_samples, 9) for 9 quantiles

# Convert y_test_tensor to numpy for easier comparison
y_true = y_test_tensor.numpy()

# Extract the 50% quantile from predictions (assuming it's the 5th quantile at index 4)
y_pred = y_pred_tensor[:, 4].numpy()

# Now y_true contains the true test values and y_pred contains the 50% quantile predictions


In [82]:
len(y_pred)

8634

In [83]:
len(y_true_np)

8634

In [35]:
import torch
import plotly.graph_objects as go

# Assuming your trained model is called 'model'

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation (since we're only making predictions)
with torch.no_grad():
    # Get model predictions for X_test_tensor
    y_pred_tensor = model(X_test_tensor)  # Shape: (num_samples, 9) for 9 quantiles

# Convert y_test_tensor to numpy for easier comparison
y_true = y_test_tensor.numpy().flatten()  # Flatten to 1D array for plotting

# Extract the 50% quantile from predictions (assuming it's the 5th quantile at index 4)
y_pred = y_pred_tensor[:, 4].numpy().flatten()  # Flatten to 1D array for plotting

# Now y_true contains the true test values and y_pred contains the 50% quantile predictions

# Create the line plot using Plotly
fig = go.Figure()

# Add real values (y_true) to the plot
fig.add_trace(go.Scatter(x=list(range(len(y_true))), 
                         y=y_true, 
                         mode='lines+markers', 
                         name='True Values'))

# Add 50% quantile predictions (y_pred) to the plot
fig.add_trace(go.Scatter(x=list(range(len(y_pred))), 
                         y=y_pred, 
                         mode='lines+markers', 
                         name='50% Quantile Predictions'))

# Update layout
fig.update_layout(title='True Values vs 50% Quantile Predictions',
                  xaxis_title='Index',
                  yaxis_title='Value',
                  template='plotly')

# Show the plot
fig.show()


In [ ]:
# Plot the results with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, num_epochs+1)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=list(range(1, num_epochs+1)), y=test_losses, mode='lines', name='Test Loss'))

fig.update_layout(title='Training and Test Loss Over Epochs',
                  xaxis_title='Epoch',
                  yaxis_title='Loss')

fig.show()

In [36]:
# Plot the predictions vs. true targets with Plotly
y_pred_numpy = y_pred
y_test_numpy = y_test_tensor.numpy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test_numpy.flatten(), y=y_pred_numpy.flatten(), mode='markers', name='Predictions'))

fig.update_layout(title='Predictions vs. True Targets',
                  xaxis_title='True Targets',
                  yaxis_title='Predictions')

fig.show()

In [37]:
# average error
np.mean(np.abs(y_pred_numpy.flatten() - y_test_numpy.flatten()))

21.375593

In [38]:
2597.3718/512

5.072991796875

In [39]:
mae = np.mean(np.abs(y_true - y_pred))
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Create scatter plot (y_true vs y_pred)
fig = go.Figure()

# Add scatter plot for true vs predicted values
fig.add_trace(go.Scatter(x=y_true, 
                         y=y_pred, 
                         mode='markers', 
                         name='Predicted vs True'))

# Add a line y = x for reference (perfect prediction line)
fig.add_trace(go.Scatter(x=y_true, 
                         y=y_true, 
                         mode='lines', 
                         name='y = x', 
                         line=dict(color='red', dash='dash')))

# Update layout
fig.update_layout(title='Scatter Plot: True Values vs Predicted (50% Quantile)',
                  xaxis_title='True Values',
                  yaxis_title='Predicted Values',
                  template='plotly')

# Show the plot
fig.show()

Mean Absolute Error (MAE): 21.3756


In [40]:
correlation_coefficient = np.corrcoef(y_true, y_pred)[0, 1]
correlation_coefficient

0.9711701322753413

In [30]:
# Save the model's state dictionary
model_path = 'model.pth'  # Choose your file path and name
torch.save(model.state_dict(), model_path)